In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import sys
import pandas as pd




In [77]:
dataPred = pd.read_csv("../Prediction_Files/predictions_eRNN.csv")

# reading predictions and transforming data

In [79]:
dataPred.Hours = ((dataPred.Hours)*23)+0
dataPred.Minutes = ((dataPred.Minutes)*59)+0
dataPred.Seconds = ((dataPred.Seconds)*59.999805)+0.102506

In [80]:
def rowToTime(row):
    timeAtMoment = row.Hours*3600*1000
    timeAtMoment += row.Minutes*60*1000
    timeAtMoment += row.Seconds*1000
    return timeAtMoment

In [81]:
resetAddTime = 23*3600*1000
resetAddTime += 59*60*1000
resetAddTime += 59*1000

### this is converting hours, minutes, seconds to a millisecond time series

In [82]:
cumulativeTime = [rowToTime(dataPred.iloc[0])]

for i in range(1,len(dataPred)):
    timeAtMoment = rowToTime(dataPred.iloc[i])
    lastTimeMoment = rowToTime(dataPred.iloc[i-1])
    if timeAtMoment > lastTimeMoment:
        cumuTime = cumulativeTime[i-1] + (timeAtMoment - lastTimeMoment)
        cumulativeTime.append(cumuTime)
    else:
        cumulativeTime.append(cumulativeTime[i-1]+(timeAtMoment+resetAddTime)-lastTimeMoment)

In [84]:
cumulativeTime = cumulativeTime- cumulativeTime[0]
cumulativeTime

array([0.00000000e+00, 7.61549680e+07, 8.33844191e+07, ...,
       1.61140254e+11, 1.61175596e+11, 1.61229291e+11])

In [168]:
dataPred.describe()

,Hours,Minutes,Seconds,Server,Tactic,Latency,Cost,Reliability,Monday,Tuesday,...,Thursday,Friday,Saturday,Sunday,Latency.1,Cost.1,Reliability.1,predicted_Latency,predicted_Cost,predicted_Reliability
count,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,...,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000
mean,11.235216,29.240520,29.682610,0.503921,0.505436,0.014090,0.308970,0.995757,0.125961,0.079554,...,0.178202,0.163882,0.152745,0.168125,0.014080,0.308999,0.995757,0.021096,0.386548,0.984344
std,7.176116,17.558624,17.397074,0.335570,0.353887,0.033737,0.152064,0.065008,0.331850,0.270638,...,0.382734,0.370218,0.359789,0.374027,0.033735,0.152080,0.065008,0.183227,0.151036,0.012001
min,0.000000,0.000000,0.118609,0.000000,0.000000,0.000000,0.001337,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001337,0.000000,-0.522015,-0.223116,0.896096
25%,4.999993,13.999992,14.676309,0.142857,0.250000,0.001319,0.291444,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001319,0.291444,1.000000,-0.106017,0.287273,0.980119
50%,11.000003,28.999975,29.662010,0.571429,0.500000,0.003015,0.331551,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003014,0.331551,1.000000,0.020240,0.395810,0.987926
75%,18.000007,45.000008,44.679331,0.857143,0.750000,0.021191,0.338235,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.021180,0.338235,1.000000,0.145517,0.495279,0.992548
max,23.000000,59.000000,60.097331,1.000000,1.000000,0.993189,0.796791,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.993189,0.796791,1.000000,0.534281,0.794719,0.998144


### shifting and transforming data to a range of 0-50

In [244]:
def shiftTransformCol50(data):
    return (data + abs(data.min()))*50

In [245]:
newData = np.stack((cumulativeTime, shiftTransformCol50(dataPred.Latency), shiftTransformCol50(dataPred.Cost), dataPred.Reliability, shiftTransformCol50(dataPred.predicted_Latency), shiftTransformCol50(dataPred.predicted_Cost), round(dataPred.predicted_Reliability)),axis = 1)
newData = pd.DataFrame(newData, columns=['Timestamp', 'Latency', 'Cost', 'Reliability', 'predLatency', 'predCost', 'predReliability' ])


In [246]:
newData.describe()

,Timestamp,Latency,Cost,Reliability,predLatency,predCost,predReliability
count,3.771000e+03,3771.000000,3771.000000,3771.000000,3771.000000,3771.000000,3771.0
mean,8.082821e+10,0.704504,15.515327,0.995757,27.155531,30.483183,1.0
std,4.671383e+10,1.686852,7.603199,0.065008,9.161371,7.551814,0.0
min,0.000000e+00,0.000000,0.133690,0.000000,0.000000,0.000000,1.0
25%,4.048662e+10,0.065960,14.639045,1.000000,20.799925,25.519450,1.0
50%,8.060520e+10,0.150734,16.644395,1.000000,27.112725,30.946300,1.0
75%,1.212482e+11,1.059530,16.978595,1.000000,33.376600,35.919775,1.0
max,1.612293e+11,49.659450,39.906395,1.000000,52.814800,50.891750,1.0


In [200]:
data = newData

In [202]:
def utilFunction(latency, cost, reward = 100, reliability = 1):
    epsilon = 0.00000000000000000000000000000000000000000000001

    if reliability == 0:
        return 0
    else:
        return ((reward)/(latency+cost+epsilon)) * reliability 
    


realUtilityMean = 4.138921e+06	
realUtilStd = 6.175693e+06	

In [253]:
lastUpdate = 0;
updateThreshold = 7.778549e+05
utilitPredicted = []
utilityReal = []
award = []
lostUtility = []

In [254]:
# real time utility aware analysis tool

In [255]:
temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']
# type(temp.total_seconds())
# pd.to_datetime(data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp'], unit='ms')

In [256]:
for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
    timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp'])

    predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predCost,(float)(timeDeltaReward), data.iloc[i].predReliability )
    
    realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
    utilitPredicted.append(predictedUtil)
    utilityReal.append(realUtility)
    if predictedUtil >= updateThreshold:
#         award.append(award[i-1] + realUtility-1);
        award.append(realUtility-updateThreshold);
        lostUtility.append(0)
    else:
#         award.append(award[i-1]);
        award.append(0);
    
        if realUtility >= updateThreshold:
            lostUtility.append(realUtility)
        else:
            lostUtility.append(0)


In [257]:
results = pd.DataFrame(np.stack([utilitPredicted,utilityReal,award,lostUtility],axis=1),columns=['predictedUtility','realUtility','award','lostUtility'])

In [258]:
results.describe()

,predictedUtility,realUtility,award,lostUtility
count,3.770000e+03,3.770000e+03,3.770000e+03,3.770000e+03
mean,7.778549e+05,4.138922e+06,2.585261e+06,1.127839e+06
std,5.179301e+05,6.175693e+06,6.033169e+06,2.564953e+06
min,2.482319e+01,0.000000e+00,-7.778549e+05,0.000000e+00
25%,3.541322e+05,1.275865e+06,0.000000e+00,0.000000e+00
50%,7.345436e+05,2.606793e+06,0.000000e+00,0.000000e+00
75%,1.120135e+06,4.282203e+06,3.086565e+06,1.458599e+06
max,6.062105e+06,1.947388e+08,1.939609e+08,2.446723e+07


In [259]:
results.to_csv("rewardAwareUtility-mean")